# Using fhir-query

## fq (fhir-query): Your FHIR Querying Assistant
The fq utility, short for "fhir-query," is a command-line tool specifically designed to simplify the process of interacting with FHIR servers. It provides researchers with a convenient way to:

1. Retrieve the vocabulary of a FHIR server: With the vocabulary command, fq fetches and summarizes the key data elements (CodeableConcepts and Extensions) used within the FHIR data. This creates a central vocabulary Dataframe that helps researchers identify important data elements and their usage within the server.


2. Execute queries to retrieve FHIR resources: Researchers can then use fq to execute FHIR queries using a readable syntax. This helps to retrieve and filter data from the FHIR Server based on various search parameters and criteria.

In [4]:
!pip install fhir-aggregator-client --no-cache-dir --quiet

In [5]:
!fq

Usage: fq [OPTIONS] COMMAND [ARGS]...

  Run FHIR GraphDefinition traversal.

Options:
  --help  Show this message and exit.

Commands:
  main*       Run FHIR GraphDefinition traversal.
  dataframe   Create dataframes from the local db.
  summarize   Summarize the aggregation results.
  visualize   Visualize the aggregation results.
  vocabulary  Retrieve Vocabulary Observation and ResearchStudy resources...


### Overview  

This notebook leverages **[FHIR GraphDefinition](https://hl7.org/fhir/graphdefinition.html)** objects to define and execute graph-based traversals across multiple interconnected FHIR resource graphs. The data retrieved is written to a **local SQLite database** for persistence and later transformed into **analyst-friendly dataframes** for analysis using tools like Python’s pandas library.

By using **FHIR GraphDefinition**, we declaratively define resource relationships and efficiently retrieve data. Once retrieved, the data is stored locally and can be transformed into dataframes for advanced analysis.

The [fhir-aggregator-client](https://github.com/FHIR-Aggregator/fhir-aggregator-client) tool runs an R5 GraphDefinition against a FHIR server

### Key Features  

- **GraphDefinition-Driven Traversals**: Use GraphDefinition objects to define explicit relationships between resources and automate traversal logic.  
- **Local SQLite Storage**: Persist the retrieved FHIR data in a local SQLite database for querying and offline analysis.  
- **Analyst-Friendly Dataframes**: Convert stored FHIR resources into pandas dataframes for ease of use in analytical workflows.  
- **Reusable Graph Definitions**: Maintain a library of GraphDefinition YAML files that can be reused across different workflows and projects.  Researchers and Data submitters can publish GraphDefinition files to help others navigate their data.

### Run a GraphDefinition

In [16]:
!fq --fhir-base-url https://google-fhir.fhir-aggregator.org --graph-definition-id '/Condition?code:text=cholangiocarcinoma'

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Parker.Gray\AppData\Local\Programs\Python\Python313\Scripts\fq.exe\__main__.py", line 7, in <module>
    sys.exit(cli.cli())
             ~~~~~~~^^
  File "C:\Users\Parker.Gray\AppData\Local\Programs\Python\Python313\Lib\site-packages\click\core.py", line 1161, in __call__
    return self.main(*args, **kwargs)
           ~~~~~~~~~^^^^^^^^^^^^^^^^^
  File "C:\Users\Parker.Gray\AppData\Local\Programs\Python\Python313\Lib\site-packages\click\core.py", line 1082, in main
    rv = self.invoke(ctx)
  File "C:\Users\Parker.Gray\AppData\Local\Programs\Python\Python313\Lib\site-packages\click\core.py", line 1697, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
                           ~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^
  File "C:\Users\Parker.Gray\AppData\Local\Programs\Python\Python313\Lib\site-packages\click\core.py",

#### Analyse Results

The graph represents relationships between different FHIR resources.Examples of FHIR resources include Patient, Condition, Observation, Procedure, etc.

Each node is labled as: <resource_type\>/<count\> the number of records of that type retrieved.

The edges in the graph are weighted.  The thicker the line, the more connections there are between nodes.

In [ ]:
# Create a graph of the results
!fq results visualize

In [ ]:
# Read the locally stored HTML file containing a graph visualization and displaying it within the Jupyter notebook.

from IPython.display import HTML
with open('fhir-graph.html', 'r') as file:
    html_content = file.read()

# Set the display height (in pixels)
display(HTML("<div style='height: 800px;'>{}</div>".format(html_content)))

#### Create a dataframe of results

In [ ]:
!fq results dataframe

In [ ]:
import pandas as pd 
df = pd.read_csv('fhir-graph.tsv', sep='\t')
df

### Other servers

You can use the `fq` tool with other FHIR servers.  For example, this query retrieves a study from `dbGAP`.

In [ ]:
# delete the previous results, start with a fresh database
!rm ~/.fhir-aggregator/fhir-graph.sqlite
!fq run  --fhir-base-url https://dbgap-api.ncbi.nlm.nih.gov/fhir-jpa-pilot/x1  research-study-link-iterate  '/ResearchStudy?_id=phs001232'

In [ ]:
# use the same commands to analyse results
!fq results visualize

In [ ]:
# create a graph of the results

from IPython.display import HTML
with open('fhir-graph.html', 'r') as file:
    html_content = file.read()

# Set the display height (in pixels)
display(HTML("<div style='height: 800px;'>{}</div>".format(html_content)))

In [ ]:
# create a dataframe of results
!fq results dataframe

In [ ]:
import pandas as pd

df = pd.read_csv('fhir-graph.tsv', sep='\t')

df